In [ ]:
import sys
sys.path.append("../..")
import pickle
import numpy as np

from SO3.movement_data.calculate_reparameterized_distance import reparameterized_distance
from my_help_functions import tictoc
from SO3.utils.io_utils import load_movements, save_results

from SO3.utils.multiple_curves_utils import move_several_rotations_origins_to_identity, create_parameterization_several_rotations, SRVT_multiple_rotations, skew_matrix_to_vector_several_rotations
from SO3.utils.reparameterization_utils import find_optimal_diffeomorphism, reparameterize_multiple_rotations, L2_metric, local_cost

import numpy as np
from scipy.spatial.transform import Rotation
from SO3.utils.curve_utils import move_rotation_origin_to_identity, create_parameterization_rotation, SRVT_single_rotation, skew_matrix_to_vector_single_rotation, exp_map
from SO3.utils.reparameterization_utils import reparameterize_rotation

from SO3.utils.reparameterization_utils import create_shared_parameterization, local_cost, dynamic, reconstruct
import functools

In [ ]:
def local_cost_reg(k, l, i, j, q0, q1, I, lambda_, type): 
    cost_l2 = local_cost(k, l, i, j, q0, q1, I)
    new_q1 = np.sqrt((I[j]-I[l])/(I[i]-I[k]))*q1[l:j] 
    
    if type != 'L1': 
        raise ValueError("Only L1 is supported")
    
    L1_reg = lambda_ * np.sum(np.abs(new_q1))
    return cost_l2 + L1_reg 

def find_optimal_diffeomorphism_reg(q0, q1, I0, I1, depth, lambda_, type = 'L1'):
    I, q0_new, q1_new = create_shared_parameterization(q0, q1, I0, I1)
    M = I.shape[0]
    local_cost_partial = functools.partial(local_cost_reg, q0 = q0_new, q1 = q1_new, I = I, lambda_ = lambda_, type = type)

    pointers, A = dynamic(local_cost_partial, M, depth)
    path = reconstruct(pointers, M-1, M-1)

    #Construct reparametrization
    x = np.array([p[0] for p in path])/float(M-1)
    y = np.array([p[1] for p in path])/float(M-1)

    I_new = np.interp(I1, x, y)
    return I_new


In [ ]:
def c_to_q(movement):     
    moved_movement = move_several_rotations_origins_to_identity(movement)
    I = create_parameterization_several_rotations(moved_movement)
    SRVT = SRVT_multiple_rotations(I, moved_movement)
    q = skew_matrix_to_vector_several_rotations(SRVT)
    return I, q

def reparameterize(q0, q1, c1, I0, I1, depth): 
    I_new = find_optimal_diffeomorphism(q0, q1, I0, I1, depth = depth)
    c_new = reparameterize_multiple_rotations(I_new, I1, c1)
    _, q_new = c_to_q(c_new)
    return c_new, q_new, I_new

def reparameterize_reg(q0, q1, c1, I0, I1, depth, lambda_, type = 'L1'):
    I_new = find_optimal_diffeomorphism_reg(q0, q1, I0, I1, depth = depth, lambda_ = lambda_, type = type)
    c_new = reparameterize_multiple_rotations(I_new, I1, c1)
    _, q_new = c_to_q(c_new)
    return c_new, q_new, I_new

def load_movements(): 
    with open('pickle_data/movements.pkl', 'rb') as f:
        movements = pickle.load(f)
    return movements

movements = load_movements()
c0 = movements['16_05.amc']['curve']
c1 = movements['16_06.amc']['curve']
c2 = movements['16_36.amc']['curve']

In [ ]:
I0, q0 = c_to_q(c0)
I1, q1 = c_to_q(c1)
I2, q2 = c_to_q(c2)

In [ ]:
c1_new, q1_new, I_new = reparameterize(q0, q1, c1, I0, I1, 3)
c2_new, q2_new, I_new = reparameterize(q0, q2, c2, I0, I2, 3)

In [ ]:
for lam in [0.005, 0.001]:
    c1_new_reg, q1_new_reg, I_new_reg = reparameterize_reg(q0, q1, c1, I0, I1, 3, lam)
    print(f"Reg: {L2_metric(q0, q1_new_reg, I0, I1)}")

In [ ]:
print(L2_metric(q0, q1, I0, I1))
print(f"Std: {L2_metric(q0, q1_new, I0, I1)}")
print(f"Reg: {L2_metric(q0, q1_new_reg, I0, I1)}")

print("")

print(L2_metric(q0, q2, I0, I2))
print(L2_metric(q0, q2_new, I0, I2))



### Create syntetic data 
- create a curve c_1 in SO3
- create a representation of c_2
- add noise to c_1 and c_2 s.t. they still are SO3 elements

In [ ]:
def random_rotation(epsilon):
    """
    Generates a random rotation matrix in SO(3) with a small perturbation.

    Args:
        epsilon: A small value controlling the magnitude of the perturbation.

    Returns:
        A rotation matrix representing a perturbed element in SO(3).
    """
    # Generate random axis
    axis = np.random.rand(3)
    axis /= np.linalg.norm(axis)

    # Scale angle with epsilon
    theta = np.random.normal(0, epsilon)

    # Construct rotation matrix with Rodrigues' formula
    R_perturb = Rotation.from_rotvec(axis * theta).as_matrix()

    return R_perturb

def is_SO3(c, tol = 1e-6):
    assert c.shape == (3, 3), "Shape is not correct"
    if not np.allclose(c.T @ c, np.eye(3), atol=tol):
        return False 
    if not np.isclose(np.linalg.det(c), 1, atol=tol):
        return False
    
    return True

def create_syntetic_curve(c_0, rho_hat, I):
    """
    Create a synthetic curve

    Input: 
        c_0: The initial rotation
        rho_hat: A function that takes a parameter and returns a skew-symmetric matrix, i.e. an element of so(3)
        I: The parameter values [0,...,1]

    Output:
        c: The curve in SO(3)
    """
    c = np.zeros((I.shape[0], 3, 3))
    for i, t in enumerate(I): 
        c[i] = c_0 @ exp_map(rho_hat(t))
        assert is_SO3(c[i]), f"Element {i} is not in SO(3)"
    return c

def rho_hat(t):
    """
    Create a skew-symmetric matrix in so(3) given a parameter t

    Input: 
        t: The parameter value

    Output:
        rho_hat: A skew-symmetric matrix (3x3)
    """
    w_0 = np.exp(t / 10)
    w_1 = np.sin(t)
    w_2 = np.cos(t)
    return np.array([[0, -w_2, w_1], [w_2, 0, -w_0], [-w_1, w_0, 0]])

def perturb_SO3_curve(c, epsilon): 
    for i in range(c.shape[0]):
        c[i] = c[i] @ random_rotation(epsilon)
    return c

functions = {
    'identity': lambda t: t,
    "sin": lambda t: np.sin(t) / np.sin(1),
    "square": lambda t: t**2,
    "log": lambda t: np.log(t + 1) / np.log(2),
    "sqrt": lambda t: np.sqrt(t),
    "exp": lambda t: (np.exp(t) - 1) / (np.exp(1) - 1),
}

def c_to_q_single(movement):     
    moved_movement = move_rotation_origin_to_identity(movement)
    I = create_parameterization_rotation(moved_movement)
    SRVT = SRVT_single_rotation(I, moved_movement)
    q = skew_matrix_to_vector_single_rotation(SRVT)
    return I, q

def reparameterize_single(q0, q1, c1, I0, I1, depth): 
    I_new = find_optimal_diffeomorphism(q0, q1, I0, I1, depth = depth)
    c_new = reparameterize_rotation(I_new, I1, c1)
    _, q_new = c_to_q_single(c_new)
    return c_new, q_new, I_new

In [ ]:
def reparameterize_single_reg(q0, q1, c1, I0, I1, depth, lambda_, type = 'L1'): 
    I_new = find_optimal_diffeomorphism_reg(q0, q1, I0, I1, depth = depth, lambda_= lambda_, type = type)
    c_new = reparameterize_rotation(I_new, I1, c1)
    _, q_new = c_to_q_single(c_new)
    return c_new, q_new, I_new



In [ ]:
# Set seed 
# np.random.seed(0)

c_0 = np.eye(3)
phi = functions['sin']
epsilon = 0.01

n_timesteps = 100
I = np.linspace(0, 1, n_timesteps)
c0 = create_syntetic_curve(c_0, rho_hat, phi(I))
c1 = create_syntetic_curve(c_0, rho_hat, I)

print(np.linalg.norm(c0 - c1))

c0_p = perturb_SO3_curve(c0, epsilon)
c1_p = perturb_SO3_curve(c1, epsilon)

print(np.linalg.norm(c0_p - c1_p))


In [ ]:
I0, q0 = c_to_q_single(c0)
I1, q1 = c_to_q_single(c1)
I0_p, q0_p = c_to_q_single(c0_p)
I1_p, q1_p = c_to_q_single(c1_p)

# c1_new, q1_new = reparameterize_single(q0, q1, c1, I0, I1, 3)

depth = 10
c1_p_new, q1_p_new, I1_p_new = reparameterize_single(q0_p, q1_p, c1_p, I0_p, I1_p, depth)

In [ ]:
c1_p_new_reg, q1_p_new_reg, I1_p_new_reg = reparameterize_single_reg(q0_p, q1_p, c1_p, I0_p, I1_p, depth, 0.02)

In [ ]:
import matplotlib.pyplot as plt

print(L2_metric(q0_p, q1_p, I0, I1))
print(f"Std dist: {L2_metric(q0, q1_p_new, I0, I1)}")
print(f"Reg dist: {L2_metric(q0, q1_p_new_reg, I0, I1)}")

plt.plot(I, phi(I), label = 'phi(t)', c = 'g')
plt.plot(I, I1_p_new, label = 'rho(t) std', c = 'b')
plt.plot(I, I1_p_new_reg, label = 'rho(t) reg', c = 'r')

plt.legend()
plt.show()